In [1]:
from keras.datasets import cifar10
from keras.utils import to_categorical

# I begin by loading up the cifar10 dataset and modifying it to be used
# in the nerual network I will later train
# I do this now because it is quite simple to get this out of the way all at once

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


This section is for the designing the neural network.

I am currently experimenting with branching neural networks to see how they affect the effectiveness of neural networks. At the same time, there are some things that I've read in François Collet's book. I want to try experimenting with 1x1 Conv nets. These apparently have some good results on nets.

In [ ]:
from keras.layers import Dense, Dropout, Flatten # Fully connected layer functions
from keras.layers import MaxPooling2D, Conv2D # Convulution specific layers
from keras.layers import Input, concatenate # Functional API specific layers
from keras.models import Sequential # For compiling the final model
from keras.layers import BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = (32, 32, 3), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'rmsprop',
             metrics = ['accuracy'])

from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks_list = [
    EarlyStopping(monitor = 'val_loss', patience = 2),
    ModelCheckpoint('cifar10_branched_Conv1.h5', save_best_only = True)
]

history = model.fit(x_train, y_train,
                   epochs = 50,
                   batch_size = 32,
                   callbacks = callbacks_list,
                   validation_data = (x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
